<a href="https://colab.research.google.com/github/ecloguehwang/HSS/blob/master/%EA%B5%AC%EC%A1%B0%EB%B0%A9%EC%A0%95%EC%8B%9D%EB%AA%A8%ED%98%95_0520v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#구조방정식모형 코드: 수정버전(2023.05.20v)

#리스트 비우기
rm(list=ls())

#### 1. 분석 준비 ####
pacman::p_load(ggpubr, ggpmisc, gridExtra,
               readxl, reshape2, FSA, dlookr, magrittr,                        # 데이터 전처리
               ShinyItemAnalysis, mirt, sirt, difNLR, difR,                    # 문항분석
               psych, GPArotation, PerformanceAnalytics, ggcorrplot,                       # 신뢰도 및 탐색적 요인분석 등
               lavaan, semTools, fastDummies,                                  # 구조방정식
               car, rstatix, agricolae, effectsize, rcompanion, ggstatsplot,   # 기타 통계분석
               tidyverse)     




#### 2. 데이터 로딩 ####
df <- fit <- result <- list()
(data <- df$rdata_인터넷사용 <- read_excel("rdata.xlsx") %>% dplyr::select(q14:q53))
(data <- df$rdata_학습태도 <- read_excel("rdata.xlsx") %>% dplyr::select(q54:q58))
(data <- df$rdata_사회적관계 <- read_excel("rdata.xlsx") %>% dplyr::select(q59:q76))
(data <- df$rdata_삷의질 <- read_excel("rdata.xlsx") %>% dplyr::select(q77:q100))
(data <- df$rdata_자기통제력 <- read_excel("rdata.xlsx") %>% dplyr::select(q101:q125))
(data <- df$rdata_스트레스 <- read_excel("rdata.xlsx") %>% dplyr::select(q126:q148))



#### 가. 1차원성 ####
DETECT1 <- conf.detect(data.frame(data), score=rowSums(data), itemcluster=rep(0, ncol(data)))
DETECT1$detect.summary %>% rownames_to_column("indices") %>% dplyr::select(1:2) %>% 
  mutate(unweighted=round(unweighted, 2))
# 0.2보다 작으므로 1차원성을 만족



#### 나. 문항적합도 ####
item <- data 
fit_item <- mirt(item, model=1, itemtype=c(rep('graded', ncol(item))), verbose = F, SE = TRUE)

dt <- fit_item %>% mirt::itemfit(fit_stats = "infit") %>% dplyr::select(item, outfit, infit) %>% 
  rownames_to_column("id") %>% mutate(id=as.numeric(id)) %>% 
  mutate_at(vars(outfit, infit), ~round(., 3))
clipr::write_clip(dt)
max <- max(dt$infit)
len <- nrow(dt)

dt %>% pivot_longer(-c(id, item)) %>% 
  ggplot(aes(value, reorder(item, desc(id)), color=name)) + 
  theme_app() + geom_point(shape=17, size=3) +
  ggplot2::annotate("rect", xmin = 0.6, xmax = 1.4, ymin = 0, ymax = len+0.6, alpha = .2, fill = "green") +
  scale_x_continuous(limits = c(0, ifelse(max<=2, 2, max))) + 
  scale_color_manual(values=c("tomato", "royalblue"), labels=c("내적합도", "외적합도")) +
  theme(legend.position=c(0.16, 0.84)) +
  labs(y=NULL, x="적합도")


#### 나. 난이도 변별도 ####
result0 <- dlookr::describe(item) %>% rename(item=1) %>% 
  rename(min=p00, median=p50, max=p100) %>% 
  dplyr::select(item, n, mean, sd, min, median, max, skewness, kurtosis) %>% 
  mutate_at(vars(mean, sd, min, median, max, skewness, kurtosis), ~round(., 2))


result1 <- ItemAnalysis(item, k=3, l=1, u=3) %>% rownames_to_column("item") %>% 
  rename(난이도=2, 변별도=ULI) %>% 
  mutate_at(vars(Mean, SD, Min.score, Max.score, 변별도), ~as.numeric(sprintf("%.2f", .))) %>% 
  mutate_at(vars(RIT, Alpha.drop), ~as.numeric(sprintf("%.3f", .))) %>% 
  mutate(난이도=round(난이도, 3)) %>% 
  mutate(난이도구분=case_when(난이도<1 & 난이도>=0.8~"매우 쉬움", 난이도<0.8 & 난이도>=0.6~"쉬움", 
                         난이도>=0.2 & 난이도<0.4~"어려움", 난이도>0 & 난이도<0.2~"매우 어려움",  
                         난이도<0.6 & 난이도>=0.4~"적절함", is.na(난이도)~"부적절함",
                         TRUE~"부적절함")) %>% 
  mutate(변별도구분=case_when(변별도>=0.8~"매우 높음", 변별도<0.8 & 변별도>=0.4~"높음", 
                         변별도>=0.3 & 변별도<0.4~"적절함", 변별도>=0.2 & 변별도<0.3~"낮음",  
                         변별도>=0 & 변별도<0.2~"거의 없음",TRUE~"부적절함"))


result2 <- coef(fit_item, IRTpars=T, verbose=F, simplify=T, order=T) %>% .[1] %>% data.frame() %>% 
  mutate(irt난이도=rowMeans(dplyr::select(., -1), na.rm=T)) %>% 
  mutate(irt난이도=ifelse(irt난이도<(-2.5), 2.5, irt난이도)) %>% 
  mutate(irt난이도구분=case_when(irt난이도<(-2) ~ "매우 쉬움",
                            irt난이도>=(-2) & irt난이도<(-0.5) ~ "쉬움",
                            irt난이도>=(-0.5) & irt난이도<0.5 ~ "적절함",
                            irt난이도>=0.5 & irt난이도<2 ~ "어려움",
                            TRUE ~ "매우 어려움")) %>% 
  rownames_to_column("item") %>% rename(irt변별도=2) %>% mutate(irt변별도=ifelse(irt변별도>2, 2, irt변별도)) %>% 
  mutate(irt변별도구분=case_when(irt변별도>=1.7 ~ "매우 높음",
                            irt변별도>=1.35 & irt변별도<1.7 ~ "높음",
                            irt변별도>=0.65 & irt변별도<1.35 ~ "적절함",
                            irt변별도>=0.35 & irt변별도<0.65 ~ "낮음",
                            irt변별도>=0.01 & irt변별도<0.35 ~ "거의 없음",
                            TRUE ~ "부적절함")) %>% as_tibble()

result_item <- result0 %>% left_join(result1 %>% dplyr::select(item, 난이도, 난이도구분, 변별도, 변별도구분, RIT, Alpha.drop)) %>%  
  left_join(result2 %>% dplyr::select(item, irt난이도, irt난이도구분, irt변별도, irt변별도구분)) %>% 
  rownames_to_column("id") %>% mutate(id=as.numeric(id)) %>% 
  mutate(난이도=ifelse(mean==min, 1, ifelse(mean==max, 0, 난이도)))


min <- min(result_item$변별도)

p0 <- result_item %>% mutate(temp=2) %>% 
  ggplot(aes(1, reorder(item, desc(id)), fill="")) + 
  geom_col(aes(temp, reorder(item, desc(id)))) +
  geom_text(aes(label=item)) + 
  theme(legend.position = "bottom",
        legend.title = element_text(color = 'white')) +
  scale_fill_manual(values=c("white")) +
  geom_segment(aes(x=0.94,xend=0.96,yend=item)) +
  geom_segment(aes(x=1.04,xend=1.065,yend=item)) +
  ylab(NULL) + ggtitle("") + 
  scale_x_continuous(expand=c(0,0), limits=c(0.94, 1.065)) +
  theme(axis.title=element_blank(),
        panel.grid=element_blank(),
        axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        panel.background=element_blank(),
        plot.title = element_text(hjust = 0.5),
        axis.text.x=element_text(color=NA),
        axis.ticks.x=element_line(color=NA),
        plot.margin = unit(c(1,-1,1,-1), "mm"))+
  labs(x="") + guides(fill=guide_legend(nrow=2, byrow=TRUE))

p0 <- ggplot_build(p0) %>% ggplot_gtable()


#dev.off()

p1 <- result_item %>% mutate(난이도구분=factor(난이도구분, levels=c("매우 쉬움", "쉬움", "적절함", "어려움", "매우 어려움"))) %>% 
  ggplot(aes(난이도, reorder(item, desc(id)), fill=난이도구분)) + 
  geom_col() + # theme_app() + 
  theme(legend.position = "bottom",
        legend.title = element_blank()) +
  geom_text(aes(x=난이도, label=sprintf("%.2f", 난이도)), 
            position=position_dodge(width=0.8), hjust=-0.2, size = 5) +
  theme(axis.title.x = element_blank(), 
        axis.title.y = element_blank(), 
        axis.text.y = element_blank(), 
        axis.ticks.y = element_blank(),
        panel.background=element_blank(),
        plot.title = element_text(hjust = 0.5),
        plot.margin = unit(c(1,0,1,0), "mm")) +
  scale_fill_brewer(palette = "Pastel1") +
  scale_x_reverse() + ggtitle("CTT 난이도") + guides(fill=guide_legend(nrow=2, byrow=TRUE))


p2 <- result_item %>% 
  mutate(irt변별도구분=factor(irt변별도구분, levels=c("부적절함", "거의 없음", "낮음", "적절함", "높음", "매우 높음"))) %>% 
  ggplot(aes(irt변별도, reorder(item, desc(id)), fill=irt변별도구분)) + 
  geom_col() + # theme_app() + 
  theme(legend.position = "bottom",
        legend.title = element_blank()) +
  geom_text(aes(x=ifelse(irt변별도>0.45, irt변별도, 0.45), label=sprintf("%.2f", irt변별도)), 
            position=position_dodge(width=0.8), hjust=1.1, size = 5) +
  theme(axis.title.x = element_blank(), 
        axis.title.y = element_blank(), 
        axis.text.y = element_blank(), 
        axis.ticks.y = element_blank(),
        panel.background=element_blank(),
        plot.title = element_text(hjust = 0.5),
        plot.margin = unit(c(1,0,1,-1), "mm")) + 
  scale_fill_brewer(palette = "Pastel1") +
  ggtitle("IRT 변별도") + 
  guides(fill=guide_legend(nrow=2, byrow=TRUE))


#q126-148 문항의 CTT난이도 및 IRT변별도
grid.arrange(p1, p0, p2, ncol=3, widths=c(4/9,1/9,4/9))


result2 <- coef(fit_item, IRTpars=T, verbose=F, simplify=T, order=T) %>% .[1] %>% data.frame() %>% 
  mutate(irt난이도=rowMeans(dplyr::select(., -1), na.rm=T)) %>% 
  mutate(irt난이도=ifelse(irt난이도<(-2.5), 2.5, irt난이도)) %>% 
  mutate(irt난이도구분=case_when(irt난이도<(-2) ~ "매우 쉬움",
                            irt난이도>=(-2) & irt난이도<(-0.5) ~ "쉬움",
                            irt난이도>=(-0.5) & irt난이도<0.5 ~ "적절함",
                            irt난이도>=0.5 & irt난이도<2 ~ "어려움",
                            TRUE ~ "매우 어려움")) %>% 
  rownames_to_column("item") %>% rename(irt변별도=2) %>% #mutate(irt변별도=ifelse(irt변별도>2, 2, irt변별도)) %>% 
  mutate(irt변별도구분=case_when(irt변별도>=1.7 ~ "매우 높음",
                            irt변별도>=1.35 & irt변별도<1.7 ~ "높음",
                            irt변별도>=0.65 & irt변별도<1.35 ~ "적절함",
                            irt변별도>=0.35 & irt변별도<0.65 ~ "낮음",
                            irt변별도>=0.01 & irt변별도<0.35 ~ "거의 없음",
                            TRUE ~ "부적절함")) %>% as_tibble()

result_item <- result0 %>% left_join(result1 %>% dplyr::select(item, 난이도, 난이도구분, 변별도, 변별도구분, RIT, Alpha.drop)) %>%  
  left_join(result2 %>% dplyr::select(item, irt난이도, irt난이도구분, irt변별도, irt변별도구분)) %>% 
  rownames_to_column("id") %>% mutate(id=as.numeric(id)) %>% 
  mutate(난이도=ifelse(mean==min, 1, ifelse(mean==max, 0, 난이도)))
result_item %>% dplyr::select(item, 난이도, 변별도, irt난이도, irt변별도) %>% 
  rename(`난이도(CTT)`=2, `변별도(CTT)`=3, `난이도(IRT)`=4, `변별도(IRT)`=5) %>% 
  clipr::write_clip()


#### 다. 신뢰도 ####

#q126-q148번의 Cronbach alpha값: 클리브랜드 그래프
cbalpha <- psych::alpha(item, check.keys=T) %>% extract2(1) %>% unclass() %>% extract2(1)
min_max <- c(min(result_item$Alpha.drop), max(result_item$Alpha.drop))

result_reli <- result_item %>% mutate(alpha=cbalpha) %>%
  pivot_longer(c(alpha, Alpha.drop)) %>%
  mutate(dalpha=ifelse(value!=cbalpha, value, NA)) %>%
  mutate(name=rep(1:(n()/2), each=2)) %>%
  mutate(gb=ifelse(value>cbalpha, "bad", "good"))

cbalpha <- result_reli %>% filter(is.na(dalpha)) %>% pull(value) %>% unique()
min_max <- c(min(result_reli$dalpha, na.rm=T), max(result_reli$dalpha, na.rm=T))

x_range <- min_max[2]+0.01 - min_max[1]-0.01
#차이가 큰 클리브랜드 그래프
result_reli %>% 
  ggplot(aes(value, reorder(item, desc(id)))) +
  theme_app() +
  geom_vline(xintercept=cbalpha, linetype=2, color="tomato", linewidth=1) +
  geom_line(aes(group=name, colour=gb)) +
  ggplot2::annotate("text", x = cbalpha+0.004, y = nrow(result_reli)/2-1, size=5,
                    label = paste0(sprintf("%.3f", cbalpha))) + # "평균=", 
  geom_point(aes(x=dalpha, color=gb), size=3) +
  scale_x_continuous(limits=c(min_max[1]-0.01, ifelse(min_max[2]>0.5, min_max[2]+0.01, min_max[2]+0.25))) +
  labs(y=NULL, x="Cronbach's alpha")



#차이가 적은 클리브랜드 그래프
result_reli %>% 
  ggplot(aes(value, reorder(item, desc(id)))) +
  theme_app() +
  geom_vline(xintercept=cbalpha, linetype=2, color="tomato", linewidth=1) +
  geom_line(aes(group=name, colour=gb)) +
  ggplot2::annotate("text", x = cbalpha+0.02, y = nrow(result_reli)/2-1, size=5,
                    label = paste0(sprintf("%.3f", cbalpha))) + # "평균=", 
  geom_point(aes(x=dalpha, color=gb), size=3) +
  scale_x_continuous(limits=c(min_max[1]-0.03, ifelse(min_max[2]>0.5, min_max[2]+0.1, min_max[2]+0.25))) +
  labs(y=NULL, x="Cronbach's alpha")


result_item %>% dplyr::select(item, Alpha.drop) %>% clipr::write_clip()

item <- data
ggcorrplot::ggcorrplot(cor(item, use ="complete.obs"), p.mat=cor_pmat(item), lab=T, lab_size=3, insig="blank", digits=1)

item <- item %>% mutate_at(vars(q69:q71), ~case_when(.==5~1, .==4~2, .==3~3, .==2~4, .==1~5))

#q126-q148 상관행렬 그래프
ggcorrplot::ggcorrplot(cor(item, use ="complete.obs"), p.mat=cor_pmat(item), lab=T, lab_size=3, insig="blank", digits=1)


#### 라. 그룹별 차이 비교 ####
# library(difNLR) # dplyr::dplyr::select 와 충돌하므로 먼저 선언해야 한다.
# 서열척도
(data <- read_excel("rdata.xlsx") %>% dplyr::dplyr::select(q1, q14:q53))
(fit <- difORD(
  Data = dplyr::dplyr::select(data, -q1), group = data$q1, focal.name = 2, model = "cumulative", # adjacent
  type = "both", match = "zscore", p.adjust.method = "holm", purify = F, 
))
#q18번 그래프?
plot(fit, item = "q18", plot.type = "cumulative", group.names = c("남자", "여자"))
plot(fit, item = "q18", plot.type = "category")

coef(fit, SE = TRUE, simplify = TRUE)
coef(fit, SE = TRUE, IRTpars = TRUE, CI = 0)[["q18"]]
coef(fit, SE = TRUE, IRTpars = FALSE, CI = 0)[["q18"]]

# Generalized logistic regression(3PLcg) 정답 여부
(data <- read_excel("rdata.xlsx") %>% dplyr::dplyr::select(q14:q53, q1) %>% 
    mutate_at(vars(q14:q53), ~ifelse(.>=4, 1, 0)))
(x <- difNLR(dplyr::dplyr::select(data, -q1), group = data$q1, focal.name = 2, model = "3PLcg", match = "zscore"))
(x <- difNLR(dplyr::dplyr::select(data, -q1), group = data$q1, focal.name = 2, model = "2PL", type="nudif"))
plot(x, item = "q18")
plot(x, item = "q18", group.names = c("남자", "여자"))
plot(x, item = "q18", draw.empirical = T, draw.CI = TRUE, group.names = c("남자", "여자"))
plot(x, item = "q21", draw.empirical = T, draw.CI = TRUE, group.names = c("남자", "여자"))
plot(x, plot.type = "stat")
coef(x, SE = TRUE, simplify = TRUE)


(data <- read_excel("rdata.xlsx") %>% dplyr::dplyr::select(q14:q53, q1) %>% 
    mutate_at(vars(q14:q53), ~ifelse(.>=4, 1, 0)) %>% 
    mutate(q1=ifelse(q1==2, 0, 1)) %>%  # 그룹은 0과 1이어야 한다. 0이 여자, 1이 남자이다.
    data.frame())
(fit_logistics <- difLogistic(
  Data = dplyr::dplyr::select(data, -q1), group = data$q1, focal.name = 0, match = "score",
  type = "both", p.adjust.method = "none", purify = FALSE
))
plotDIFLogistic(fit_logistics, item = "q18", Data = dplyr::dplyr::select(data, -q1), group = data$q1, group.names = c("남자", "여자"))
fit_logistics$logitPar


# delta scores with normal threshold
library(deltaPlotR)  # dplyr::dplyr::select와 충돌한다.
(data <- read_excel("rdata.xlsx") %>% dplyr::dplyr::select(q14:q53, q1) %>% 
    mutate_at(vars(q14:q53), ~ifelse(.>=4, 1, 0)) %>% data.frame()) # data.frame 이어야 계산이 된다.
(DS_normal <- deltaPlot(
  data = data, group = "q1", focal.name = 2,
  thr = "norm", purify = T
))
# delta plot
diagPlot(DS_normal, thr.draw = TRUE, print.corr=T)
edit(diagPlot)



library(difR)
(data <- read_excel("rdata.xlsx") %>% dplyr::dplyr::select(q14:q53, q1) %>% 
    mutate_at(vars(q14:q53), ~ifelse(.>=4, 1, 0)))
# 3PL
guess <- itemParEst(dplyr::dplyr::select(data, -q1), model = "3PL")[, 3]
fitLord <- difLord(dplyr::dplyr::select(data, -q1), group = data$q1, focal.name = 1, model = "3PL",
                   c = guess, p.adjust.method = "none", purify = FALSE)
# 2PL
fitLord <- difLord(dplyr::dplyr::select(data, -q1), group = data$q1, focal.name = 1, model = "2PL",
                   p.adjust.method = "none", purify = FALSE)

rownames(fitLord$itemParInit) <- c("q14", paste0("q", 15:53), "q14", paste0("q", 15:53))
plotDIFirt(fitLord$itemParInit, item = "q18")
plotDIFirt(fitLord$itemParInit, item = "q18", same.scale=T)
plotDIFirt(fitLord$itemParInit, "Raju", item = 5) # , same.scale=T
# Reference(2)는 여자, Focal(1)은 남자이다. 남자보다 여자들이 호불호가 분명한 경향이 있다.

edit(plotDIFirt)


dev.off()
#### 3. 신뢰도와 타당도 ####
data <- read_excel("rdata.xlsx") %>% dplyr::dplyr::select(q14:q148)
paste0('q', sprintf('%d', 14:53)) %>% paste0(collapse = "+")
paste0('q', sprintf('%d', 59:76)) %>% paste0(collapse = "+")
paste0('q', sprintf('%d', 77:100)) %>% paste0(collapse = "+")
paste0('q', sprintf('%d', 101:125)) %>% paste0(collapse = "+")
paste0('q', sprintf('%d', 126:148)) %>% paste0(collapse = "+")
fit$cfa <- cfa('인터넷사용=~q14+q15+q16+q17+q18+q19+q20+q21+q22+q23+q24+q25+q26+q27+q28+q29+q30+q31+q32+q33+q34+q35+q36+q37+q38+q39+q40+q41+q42+q43+q44+q45+q46+q47+q48+q49+q50+q51+q52+q53
               학습태도=~q54+q55+q56+q57+q58
               사회적관계=~q59+q60+q61+q62+q63+q64+q65+q66+q67+q68+q69+q70+q71+q72+q73+q74+q75+q76
               삶의질=~q77+q78+q79+q80+q81+q82+q83+q84+q85+q86+q87+q88+q89+q90+q91+q92+q93+q94+q95+q96+q97+q98+q99+q100
               자기통제력=~q101+q102+q103+q104+q105+q106+q107+q108+q109+q110+q111+q112+q113+q114+q115+q116+q117+q118+q119+q120+q121+q122+q123+q124+q125
               스트레스=~q126+q127+q128+q129+q130+q131+q132+q133+q134+q135+q136+q137+q138+q139+q140+q141+q142+q143+q144+q145+q146+q147+q148
               ', data=data, std.lv=TRUE) # , se="bootstrap", bootstrap=200
result$sem <- summary(fit$cfa, standardized=T, fit.measures=T, rsq=T)

# 가. 신뢰도와 집중타당도(평균분산추출)
print(result$reliability <- reliability(fit$cfa,  return.total=T) %>% t() %>% round(3) %>% as.data.frame %>% 
        rownames_to_column("var") %>% mutate(var=str_remove(var, "0")))  # 통제변인이 빠져야한다.
# 합성신뢰도 0.7이상 확보 및 평균분산추출 0.5 이상 확보 완료(집중타당도 확보)

# 나. 지표신뢰도와 집중 타당도
print(result$convergent <- parameterEstimates(fit$cfa, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
        mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
        rename(LatentFactor=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
        mutate(LatentFactor=str_remove(LatentFactor, "0"))) 
# 지표신뢰도(표준화 적재량이 0.708이상 확보, 0.6 이상 허용), Z값이 모두 2보다 크므로 집중타당도 확보 완료

# 다. 판별타당도 
print(result$discriminant <- lavInspect(fit$cfa, what="cor.lv")[1:(nrow(result$reliability)-1),] %>% 
        as.data.frame() %>%  rownames_to_column("Construct") %>%
        cbind(Square_Root_of_AVE=sqrt(reliability(fit$cfa)["avevar",])) %>% 
        mutate_if(is.numeric, round, 3) %>% mutate(Construct=str_remove(Construct, "0")))
# 통제변수가 들어간 모형으로 판별타당도를 구하면 에러가 난다. 주의!

# 라. 모형적합도 
(result$goodness <- fitMeasures(fit$cfa, fit.measures="all") %>% data.frame() %>% rename(value=1) %>% mutate_if(is.numeric, round, 3) %>% 
    filter(grepl("chisq|df|pvalue|cfi|tli|gfi|rmsea|srmr", rownames(.))) %>% t() %>% data.frame() %>% 
    mutate(CMIN=chisq/df) %>% dplyr::select(chisq, df, pvalue, gfi, rmsea, srmr, cfi, tli, CMIN) %>% 
    rename(chisq=1, df=2, pvalue=3, GFI=4, RMSEA=5, SRMR=6, CFI=7, TLI=8))
# 절대적합지수(chisq, gfi, agfi, rmr, rmsea, srmr) gfi는 모형이 자료를 얼마나 잘 설명하는지 분석하는 회귀분석의 r^2와 비슷한 의미이다.
# gfi, agif 모두 0.9보다 커야하고 rmr은 0.8이하이면 적절하다.
# 증분적합지수(nfi, cfi, tli) 0.9보다 커야한다.
# 간명기초적합지수(pcfi)-지원 안함., 간명표준적합지수(pnfi)는 값이 클수록 우수한데 0.6보다 커야 한다.

#### 4. 파셀링 ####
### 가. 1차 ###

# 표준화된 적재량 순서대로 정렬하기
# 1.인터넷사용(의존)
parameterEstimates(fit$cfa, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="인터넷사용") %>% arrange(Beta) 



# 2.학습태도
parameterEstimates(fit$cfa, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="학습태도") %>% arrange(Beta)


# 3.사회적관계
parameterEstimates(fit$cfa, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="사회적관계") %>% arrange(Beta)




# 4.삶의질
parameterEstimates(fit$cfa, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="삶의질") %>% arrange(Beta)



# 5.자기통제력
parameterEstimates(fit$cfa, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="자기통제력") %>% arrange(Beta)



# 6.스트레스
parameterEstimates(fit$cfa, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="스트레스") %>% arrange(Beta)


#문항묶기
data <- data %>% 
  mutate(인터넷사용P1_01=(q35+q39)/2, 인터넷사용P1_02=(q36+q27)/2, 인터넷사용P1_03=(q34+q20)/2, 인터넷사용P1_04=(q37+q30)/2,
         인터넷사용P1_05=(q51+q26)/2, 인터넷사용P1_06=(q50+q40)/2, 인터넷사용P1_07=(q53+q17)/2, 인터넷사용P1_08=(q48+q28)/2,
         인터넷사용P1_09=(q32+q18)/2, 인터넷사용P1_10=(q33+q22)/2, 인터넷사용P1_11=(q19+q45)/2, 인터넷사용P1_12=(q16+q41)/2,
         인터넷사용P1_13=(q49+q23)/2, 인터넷사용P1_14=(q15+q24)/2, 인터넷사용P1_15=(q31+q38)/2, 인터넷사용P1_16=(q42+q25)/2,
         인터넷사용P1_17=(q46+q52)/2, 인터넷사용P1_18=(q43+q44)/2, 인터넷사용P1_19=(q29+q47)/2, 인터넷사용P1_20=(q14+q21)/2) %>% 
  mutate(학습태도P1_01=(q54+q55)/2, 학습태도P1_02=(q56+q58)/2, 학습태도P1_03=(q57+q57)/2) %>% 
  mutate(사회적관계P1_01=(q61+q69)/2, 사회적관계P1_02=(q62+q71)/2, 사회적관계P1_03=(q60+q70)/2, 사회적관계P1_04=(q59+q74)/2,
         사회적관계P1_05=(q65+q76)/2, 사회적관계P1_06=(q63+q73)/2, 사회적관계P1_07=(q64+q75)/2, 사회적관계P1_08=(q67+q72)/2,
         사회적관계P1_09=(q66+q68)/2) %>% 
  mutate(삶의질P1_01=(q79+q77)/2, 삶의질P1_02=(q80+q78)/2, 삶의질P1_03=(q93+q100)/2, 삶의질P1_04=(q91+q95)/2, 삶의질P1_05=(q84+q86)/2,
         삶의질P1_06=(q92+q99)/2, 삶의질P1_07=(q97+q90)/2, 삶의질P1_08=(q94+q85)/2,  삶의질P1_09=(q96+q88)/2, 삶의질P1_10=(q82+q81)/2,
         삶의질P1_11=(q83+q89)/2, 삶의질P1_12=(q98+q87)/2) %>% 
  mutate(자기통제력P1_01=(q117+q104)/2, 자기통제력P1_02=(q121+q123)/2, 자기통제력P1_03=(q118+q105)/2, 자기통제력P1_04=(q103+q109)/2,
         자기통제력P1_05=(q120+q122)/2, 자기통제력P1_06=(q113+q106)/2, 자기통제력P1_07=(q112+q119)/2, 자기통제력P1_08=(q110+q124)/2,
         자기통제력P1_09=(q102+q125)/2, 자기통제력P1_10=(q108+q115)/2, 자기통제력P1_11=(q107+q101)/2, 자기통제력P1_12=(q114+q111)/2,
         자기통제력P1_13=(q116+q116)/2) %>% 
  mutate(스트레스P1_01=(q134+q138)/2, 스트레스P1_02=(q133+q131)/2, 스트레스P1_03=(q132+q126)/2, 스트레스P1_04=(q143+q137)/2,
         스트레스P1_05=(q136+q148)/2, 스트레스P1_06=(q129+q142)/2, 스트레스P1_07=(q130+q146)/2, 스트레스P1_08=(q147+q145)/2,
         스트레스P1_09=(q139+q140)/2, 스트레스P1_10=(q127+q141)/2, 스트레스P1_11=(q135+q128)/2, 스트레스P1_12=(q144+q144)/2)



#### 나. 2차 ####
paste0('인터넷사용P1_', sprintf('%02d', 1:20)) %>% paste0(collapse = "+")
paste0('학습태도P1_', sprintf('%02d', 1:3)) %>% paste0(collapse = "+")
paste0('사회적관계P1_', sprintf('%02d', 1:9)) %>% paste0(collapse = "+")
paste0('삶의질P1_', sprintf('%02d', 1:12)) %>% paste0(collapse = "+")
paste0('자기통제력P1_', sprintf('%02d', 1:13)) %>% paste0(collapse = "+")
paste0('스트레스P1_', sprintf('%02d', 1:12)) %>% paste0(collapse = "+")
fit$cfa1 <- cfa('인터넷사용=~인터넷사용P1_01+인터넷사용P1_02+인터넷사용P1_03+인터넷사용P1_04+인터넷사용P1_05+인터넷사용P1_06+인터넷사용P1_07+인터넷사용P1_08+인터넷사용P1_09+인터넷사용P1_10+인터넷사용P1_11+인터넷사용P1_12+인터넷사용P1_13+인터넷사용P1_14+인터넷사용P1_15+인터넷사용P1_16+인터넷사용P1_17+인터넷사용P1_18+인터넷사용P1_19+인터넷사용P1_20
               학습태도=~학습태도P1_01+학습태도P1_02+학습태도P1_03
               사회적관계=~사회적관계P1_01+사회적관계P1_02+사회적관계P1_03+사회적관계P1_04+사회적관계P1_05+사회적관계P1_06+사회적관계P1_07+사회적관계P1_08+사회적관계P1_09
               삶의질=~삶의질P1_01+삶의질P1_02+삶의질P1_03+삶의질P1_04+삶의질P1_05+삶의질P1_06+삶의질P1_07+삶의질P1_08+삶의질P1_09+삶의질P1_10+삶의질P1_11+삶의질P1_12
               자기통제력=~자기통제력P1_01+자기통제력P1_02+자기통제력P1_03+자기통제력P1_04+자기통제력P1_05+자기통제력P1_06+자기통제력P1_07+자기통제력P1_08+자기통제력P1_09+자기통제력P1_10+자기통제력P1_11+자기통제력P1_12+자기통제력P1_13
               스트레스=~스트레스P1_01+스트레스P1_02+스트레스P1_03+스트레스P1_04+스트레스P1_05+스트레스P1_06+스트레스P1_07+스트레스P1_08+스트레스P1_09+스트레스P1_10+스트레스P1_11+스트레스P1_12
               ', data=data, std.lv=TRUE) # , se="bootstrap", bootstrap=200
result$sem1 <- summary(fit$cfa1, standardized=T, fit.measures=T, rsq=T)

# 1.인터넷사용(의존)
parameterEstimates(fit$cfa1, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="인터넷사용") %>% arrange(Beta) 


# 2.사회적관계
parameterEstimates(fit$cfa1, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="사회적관계") %>% arrange(Beta) 


# 3.삶의질
parameterEstimates(fit$cfa1, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="삶의질") %>% arrange(Beta) 


# 4.자기통제력
parameterEstimates(fit$cfa1, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="자기통제력") %>% arrange(Beta) 


# 5.스트레스
parameterEstimates(fit$cfa1, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="스트레스") %>% arrange(Beta)

data <- data %>% 
  mutate(인터넷사용P2_01=(인터넷사용P1_06+인터넷사용P1_01)/2, 인터넷사용P2_02=(인터넷사용P1_05+인터넷사용P1_16)/2, 
         인터넷사용P2_03=(인터넷사용P1_13+인터넷사용P1_15)/2, 인터넷사용P2_04=(인터넷사용P1_02+인터넷사용P1_20)/2,
         인터넷사용P2_05=(인터넷사용P1_04+인터넷사용P1_14)/2, 인터넷사용P2_06=(인터넷사용P1_19+인터넷사용P1_18)/2,
         인터넷사용P2_07=(인터넷사용P1_12+인터넷사용P1_11)/2, 인터넷사용P2_08=(인터넷사용P1_09+인터넷사용P1_17)/2,
         인터넷사용P2_09=(인터넷사용P1_10+인터넷사용P1_03)/2, 인터넷사용P2_10=(인터넷사용P1_07+인터넷사용P1_08)/2) %>% 
  mutate(사회적관계P2_01=(사회적관계P1_08+사회적관계P1_01)/2, 사회적관계P2_02=(사회적관계P1_05+사회적관계P1_03)/2,
         사회적관계P2_03=(사회적관계P1_07+사회적관계P1_06)/2, 사회적관계P2_04=(사회적관계P1_04+사회적관계P1_02)/2,
         사회적관계P2_05=(사회적관계P1_09+사회적관계P1_09)/2) %>% 
  mutate(삶의질P2_01=(삶의질P1_09+삶의질P1_02)/2, 삶의질P2_02=(삶의질P1_05+삶의질P1_01)/2,
         삶의질P2_03=(삶의질P1_07+삶의질P1_03)/2, 삶의질P2_04=(삶의질P1_12+삶의질P1_04)/2,
         삶의질P2_05=(삶의질P1_06+삶의질P1_08)/2, 삶의질P2_06=(삶의질P1_11+삶의질P1_10)/2) %>% 
  mutate(자기통제력P2_01=(자기통제력P1_01+자기통제력P1_13)/2, 자기통제력P2_02=(자기통제력P1_03+자기통제력P1_04)/2, 
         자기통제력P2_03=(자기통제력P1_08+자기통제력P1_12)/2, 자기통제력P2_04=(자기통제력P1_09+자기통제력P1_07)/2, 
         자기통제력P2_05=(자기통제력P1_05+자기통제력P1_11)/2, 자기통제력P2_06=(자기통제력P1_10+자기통제력P1_02)/2, 
         자기통제력P2_07=(자기통제력P1_06+자기통제력P1_06)/2) %>% 
  mutate(스트레스P2_01=(스트레스P1_10+스트레스P1_12)/2, 스트레스P2_02=(스트레스P1_06+스트레스P1_09)/2,
         스트레스P2_03=(스트레스P1_07+스트레스P1_03)/2, 스트레스P2_04=(스트레스P1_05+스트레스P1_01)/2,
         스트레스P2_05=(스트레스P1_08+스트레스P1_04)/2, 스트레스P2_06=(스트레스P1_11+스트레스P1_02)/2)


#### 다. 3차 ####
paste0('인터넷사용P2_', sprintf('%02d', 1:10)) %>% paste0(collapse = "+")
paste0('사회적관계P2_', sprintf('%02d', 1:5)) %>% paste0(collapse = "+")
paste0('삶의질P2_', sprintf('%02d', 1:6)) %>% paste0(collapse = "+")
paste0('자기통제력P2_', sprintf('%02d', 1:7)) %>% paste0(collapse = "+")
paste0('스트레스P2_', sprintf('%02d', 1:6)) %>% paste0(collapse = "+")
fit$cfa2 <- cfa('인터넷사용=~인터넷사용P2_01+인터넷사용P2_02+인터넷사용P2_03+인터넷사용P2_04+인터넷사용P2_05+인터넷사용P2_06+인터넷사용P2_07+인터넷사용P2_08+인터넷사용P2_09+인터넷사용P2_10
               학습태도=~학습태도P1_01+학습태도P1_02+학습태도P1_03
               사회적관계=~사회적관계P2_01+사회적관계P2_02+사회적관계P2_03+사회적관계P2_04+사회적관계P2_05
               삶의질=~삶의질P2_01+삶의질P2_02+삶의질P2_03+삶의질P2_04+삶의질P2_05+삶의질P2_06
               자기통제력=~자기통제력P2_01+자기통제력P2_02+자기통제력P2_03+자기통제력P2_04+자기통제력P2_05+자기통제력P2_06+자기통제력P2_07
               스트레스=~스트레스P2_01+스트레스P2_02+스트레스P2_03+스트레스P2_04+스트레스P2_05+스트레스P2_06
               ', data=data, std.lv=TRUE) 
result$sem2 <- summary(fit$cfa2, standardized=T, fit.measures=T, rsq=T)

# 1.인터넷사용
parameterEstimates(fit$cfa2, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="인터넷사용") %>% arrange(Beta)



# 2.사회적관계
parameterEstimates(fit$cfa2, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="사회적관계") %>% arrange(Beta)


# 2.삶의질
parameterEstimates(fit$cfa2, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="삶의질") %>% arrange(Beta)


# 3.자기통제력
parameterEstimates(fit$cfa2, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="자기통제력") %>% arrange(Beta)


# 4.스트레스
parameterEstimates(fit$cfa2, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="스트레스") %>% arrange(Beta)


#문항묶기
data <- data %>% 
  mutate(인터넷사용P3_01=(인터넷사용P2_02+인터넷사용P2_01)/2, 인터넷사용P3_02=(인터넷사용P2_03+인터넷사용P2_09)/2,
         인터넷사용P3_03=(인터넷사용P2_05+인터넷사용P2_04)/2, 인터넷사용P3_04=(인터넷사용P2_10+인터넷사용P2_08)/2,
         인터넷사용P3_05=(인터넷사용P2_07+인터넷사용P2_06)/2) %>% 
  mutate(사회적관계P3_01=(사회적관계P2_04+사회적관계P2_05)/2, 사회적관계P3_02=(사회적관계P2_02+사회적관계P2_03)/2,
         사회적관계P3_03=(사회적관계P2_01+사회적관계P2_01)/2) %>% 
  mutate(삶의질P3_01=(삶의질P2_05+삶의질P2_06)/2, 삶의질P3_02=(삶의질P2_02+삶의질P2_03)/2,
         삶의질P3_03=(삶의질P2_04+삶의질P2_01)/2) %>% 
  mutate(자기통제력P3_01=(자기통제력P2_06+자기통제력P2_07)/2, 자기통제력P3_02=(자기통제력P2_05+자기통제력P2_01)/2,
         자기통제력P3_03=(자기통제력P2_04+자기통제력P2_03)/2, 자기통제력P3_04=(자기통제력P2_02+자기통제력P2_02)/2) %>% 
  mutate(스트레스P3_01=(스트레스P2_05+스트레스P2_06)/2, 스트레스P3_02=(스트레스P2_02+스트레스P2_01)/2,
         스트레스P3_03=(스트레스P2_04+스트레스P2_03)/2)


#### 라. 4차 ####
paste0('인터넷사용P3_', sprintf('%02d', 1:5)) %>% paste0(collapse = "+")
fit$cfa3 <- cfa('인터넷사용=~인터넷사용P3_01+인터넷사용P3_02+인터넷사용P3_03+인터넷사용P3_04+인터넷사용P3_05
               학습태도=~학습태도P1_01+학습태도P1_02+학습태도P1_03
               사회적관계=~사회적관계P3_01+사회적관계P3_02+사회적관계P2_03
               삶의질=~삶의질P3_01+삶의질P3_02+삶의질P3_03
               자기통제력=~자기통제력P3_01+자기통제력P3_02+자기통제력P3_03+자기통제력P3_04
               스트레스=~스트레스P3_01+스트레스P3_02+스트레스P3_03
               ', data=data, std.lv=TRUE) 
result$sem3 <- summary(fit$cfa3, standardized=T, fit.measures=T, rsq=T)


#인터넷사용
parameterEstimates(fit$cfa3, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
  mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
  dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
  filter(LatentFactor=="인터넷사용") %>% arrange(Beta)


#문항묶기
data <- data %>% 
  mutate(인터넷사용P4_01=인터넷사용P3_04+인터넷사용P3_01, 인터넷사용P4_02=인터넷사용P3_02+인터넷사용P3_03,
         인터넷사용P4_03=인터넷사용P3_05+인터넷사용P3_05)


#### 마. 신뢰도와 타당도 ####
fit$cfa4 <- cfa('인터넷사용=~인터넷사용P4_01+인터넷사용P4_02+인터넷사용P4_03
               학습태도=~학습태도P1_01+학습태도P1_02+학습태도P1_03
               사회적관계=~사회적관계P3_01+사회적관계P3_02+사회적관계P2_03
               삶의질=~삶의질P3_01+삶의질P3_02+삶의질P3_03
               자기통제력=~자기통제력P3_01+자기통제력P3_02+자기통제력P3_03+자기통제력P3_04
               스트레스=~스트레스P3_01+스트레스P3_02+스트레스P3_03
               ', data=data, std.lv=TRUE) 


# 가. 신뢰도와 집중타당도(평균분산추출)
print(result$reliability <- reliability(fit$cfa4,  return.total=T) %>% t() %>% round(3) %>% as.data.frame %>% 
        rownames_to_column("var") %>% mutate(var=str_remove(var, "0")))  # 통제변인이 빠져야한다.
# 합성신뢰도 0.7이상 확보 및 평균분산추출 0.5 이상 확보 완료(집중타당도 확보)
clipr::write_clip(result$reliability)

# 나. 지표신뢰도와 집중 타당도
print(result$convergent <- parameterEstimates(fit$cfa4, standardized=T) %>% filter(op=="=~")  %>% mutate_if(is.numeric, round, 3) %>% 
        mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>%  
        dplyr::select("LatentFactor"=lhs, Indicator=rhs, B=est, SE=se, Z=z, Beta=std.all, Sig.=stars) %>% 
        mutate(LatentFactor=str_remove(LatentFactor, "0"))) 
# 지표신뢰도(표준화 적재량이 0.708이상 확보, 0.6 이상 허용), Z값이 모두 2보다 크므로 집중타당도 확보 완료
clipr::write_clip(result$convergent)

# 다. 판별타당도 
print(result$discriminant <- lavInspect(fit$cfa4, what="cor.lv")[1:(nrow(result$reliability)-1),] %>% 
        as.data.frame() %>%  rownames_to_column("Construct") %>%
        cbind(Square_Root_of_AVE=sqrt(reliability(fit$cfa4)["avevar",])) %>% 
        mutate_if(is.numeric, round, 3) %>% mutate(Construct=str_remove(Construct, "0")))
clipr::write_clip(result$discriminant)
# 통제변수가 들어간 모형으로 판별타당도를 구하면 에러가 난다. 주의!

# 라. 모형적합도 
(result$goodness <- fitMeasures(fit$cfa4, fit.measures="all") %>% data.frame() %>% rename(value=1) %>% mutate_if(is.numeric, round, 3) %>% 
    filter(grepl("chisq|df|pvalue|cfi|tli|gfi|rmsea|srmr", rownames(.))) %>% t() %>% data.frame() %>% 
    mutate(`CMIN/df`=chisq/df) %>% dplyr::select(chisq, df, pvalue, gfi, rmsea, srmr, cfi, tli, `CMIN/df`) %>% 
    rename(chisq=1, df=2, pvalue=3, GFI=4, RMSEA=5, SRMR=6, CFI=7, TLI=8))
clipr::write_clip(result$goodness)



#### 5. 연구모형 단순 ####
fit$sem <- sem('인터넷사용=~인터넷사용P4_01+인터넷사용P4_02+인터넷사용P4_03
               학습태도=~학습태도P1_01+학습태도P1_02+학습태도P1_03
               사회적관계=~사회적관계P3_01+사회적관계P3_02+사회적관계P2_03
               삶의질=~삶의질P3_01+삶의질P3_02+삶의질P3_03
               자기통제력=~자기통제력P3_01+자기통제력P3_02+자기통제력P3_03+자기통제력P3_04
               스트레스=~스트레스P3_01+스트레스P3_02+스트레스P3_03
               
               자기통제력 ~ a1*인터넷사용
               스트레스 ~ a2*인터넷사용
               
               학습태도 ~ m11*자기통제력 + m12*스트레스 + n1*인터넷사용
               사회적관계 ~ m21*자기통제력 + m22*스트레스 + n2*인터넷사용
               삶의질 ~ m31*자기통제력 + m32*스트레스 + n3*인터넷사용
               
               학습태도_인터넷사용.DE:=n1
               학습태도_인터넷사용.IE:=a1*m11+a2*m12
               학습태도_인터넷사용.TE:=n1+a1*m11+a2*m12
               학습태도_자기통제력.DE:=m11
               학습태도_자기통제력.IE:=m11
               학습태도_자기통제력.TE:=m11
               학습태도_스트레스.DE:=m12
               학습태도_스트레스.IE:=m12
               학습태도_스트레스.TE:=m12
               
               사회적관계_인터넷사용.DE:=n2
               사회적관계_인터넷사용.IE:=a1*m21+a2*m22
               사회적관계_인터넷사용.TE:=n2+a1*m21+a2*m22
               사회적관계_자기통제력.DE:=m21
               사회적관계_자기통제력.IE:=m21
               사회적관계_자기통제력.TE:=m21
               사회적관계_스트레스.DE:=m22
               사회적관계_스트레스.IE:=m22
               사회적관계_스트레스.TE:=m22
               
               삶의질_인터넷사용.DE:=n3
               삶의질_인터넷사용.IE:=a1*m31+a2*m32
               삶의질_인터넷사용.TE:=n3+a1*m31+a2*m32
               삶의질_자기통제력.DE:=m31
               삶의질_자기통제력.IE:=m31
               삶의질_자기통제력.TE:=m31
               삶의질_스트레스.DE:=m32
               삶의질_스트레스.IE:=m32
               삶의질_스트레스.TE:=m32
               ', data=data, std.lv=TRUE)
result$sem <- summary(fit$sem, standardized=T, fit.measures=T, rsq=T)

(result$goodness <- fitMeasures(fit$sem, fit.measures="all") %>% data.frame() %>% rename(value=1) %>% mutate_if(is.numeric, round, 3) %>% 
    filter(grepl("chisq|df|pvalue|cfi|tli|gfi|rmsea|srmr", rownames(.))) %>% t() %>% data.frame() %>% 
    mutate(CMIN=chisq/df) %>% dplyr::select(chisq, df, pvalue, gfi, rmsea, srmr, cfi, tli, CMIN) %>% 
    rename(chisq=1, df=2, pvalue=3, GFI=4, RMSEA=5, SRMR=6, CFI=7, TLI=8)) 
clipr::write_clip(result$goodness)

print(result$regression <- result$sem$pe %>% filter(op=="~") %>% 
        dplyr::select(Dependent=lhs, Independent=rhs, Beta=std.all, Z=z, pvalue) %>% 
        mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>% 
        mutate_if(is.numeric, round, 3) %>% mutate(Dependent=str_remove(Dependent, "0")) %>% 
        mutate(Independent=str_remove(Independent, "0")))
clipr::write_clip(result$regression)

print(result$effect <- result$sem$pe %>% filter(op==":=" | op=="r2") %>% 
        mutate(stars=ifelse(pvalue < 0.001, "***",
                            ifelse(pvalue < 0.01, "**",
                                   ifelse(pvalue < 0.05, "*", "")))) %>% 
        mutate_if(is.numeric, round, 3)) 

print(result$DIeffect <- data.frame(unique(do.call(rbind.data.frame, strsplit(result$effect$label, split="\\."))[1]), 
                                    result$effect[grepl('DE', result$effect$label),] %>% dplyr::select("std.all"),
                                    result$effect[grepl('DE', result$effect$label),] %>% dplyr::select("stars"),
                                    result$effect[grepl('IE', result$effect$label),] %>% dplyr::select("std.all"),
                                    result$effect[grepl('IE', result$effect$label),] %>% dplyr::select("stars"),
                                    result$effect[grepl('TE', result$effect$label),] %>% dplyr::select("std.all"), 
                                    result$effect[grepl('TE', result$effect$label),] %>% dplyr::select("stars")) %>% 
        rename("예측변수"=1, "직접"=2, "별1"=3, "간접"=4, "별2"=5, "총"=6, "별3"=7))

result$DIeffect$직접효과 <- paste0(result$DIeffect$직접, result$DIeffect$별1)
result$DIeffect$간접효과 <- paste0(result$DIeffect$간접, result$DIeffect$별2)
result$DIeffect$총효과 <- paste0(result$DIeffect$총, result$DIeffect$별3)

print(result$DIeffect <- result$DIeffect %>% dplyr::select(예측변수, 직접효과, 간접효과, 총효과))
result$DIeffect %>% separate(예측변수, into=c("내생변수", "예측변수"), sep="_") %>% clipr::write_clip()




#### 7. 연구모형(변인통제) ####
data <- data %>% dplyr::select(인터넷사용P4_01, 인터넷사용P4_02, 인터넷사용P4_03,
                        학습태도P1_01, 학습태도P1_02, 학습태도P1_03,
                        사회적관계P3_01, 사회적관계P3_02, 사회적관계P2_03,
                        삶의질P3_01, 삶의질P3_02, 삶의질P3_03,
                        자기통제력P3_01, 자기통제력P3_02, 자기통제력P3_03, 자기통제력P3_04,
                        스트레스P3_01, 스트레스P3_02, 스트레스P3_03)
names(rdata <- read_excel("rdata.xlsx") %>% 
        rename(성별=q1, 학년=q4, 거주지=q5, 부모=q6, 부모직장생활=q7, 인터넷사용시간=q8, 인터넷사용목적=q9,
               반성적=q10, 건강상태=q11, 생활수준=q12) %>% 
        mutate_at(vars(성별, 학년, 거주지, 부모, 부모직장생활, 인터넷사용시간, 인터넷사용목적, 반성적, 건강상태, 생활수준),
                  ~as.factor(.)))

data.dummy <- dummy_cols(rdata[,c(2, 5:10, 12:14)], remove_first_dummy=T)[-c(1:10)] #  remove_first_dummy는 multicollinearity(다중공선성) 때문

data <- cbind(data, data.dummy)
strsplit(names(data.dummy), '\\s+') %>% paste(collapse="+")

fit$sem <- sem('인터넷사용=~인터넷사용P4_01+인터넷사용P4_02+인터넷사용P4_03
               학습태도=~학습태도P1_01+학습태도P1_02+학습태도P1_03
               사회적관계=~사회적관계P3_01+사회적관계P3_02+사회적관계P2_03
               삶의질=~삶의질P3_01+삶의질P3_02+삶의질P3_03
               자기통제력=~자기통제력P3_01+자기통제력P3_02+자기통제력P3_03+자기통제력P3_04
               스트레스=~스트레스P3_01+스트레스P3_02+스트레스P3_03
               
               자기통제력 ~ a1*인터넷사용+성별_2+학년_3+학년_4+학년_5+학년_6+거주지_2+거주지_3+거주지_4+부모_2+부모_3+부모_4+부모직장생활_2+부모직장생활_3+부모직장생활_4+인터넷사용시간_2+인터넷사용시간_3+인터넷사용시간_4+인터넷사용시간_5+인터넷사용목적_2+인터넷사용목적_3+인터넷사용목적_4+인터넷사용목적_5+반성적_2+반성적_3+건강상태_2+건강상태_3+건강상태_4+건강상태_5+생활수준_2+생활수준_3+생활수준_4+생활수준_5
               스트레스 ~ a2*인터넷사용+성별_2+학년_3+학년_4+학년_5+학년_6+거주지_2+거주지_3+거주지_4+부모_2+부모_3+부모_4+부모직장생활_2+부모직장생활_3+부모직장생활_4+인터넷사용시간_2+인터넷사용시간_3+인터넷사용시간_4+인터넷사용시간_5+인터넷사용목적_2+인터넷사용목적_3+인터넷사용목적_4+인터넷사용목적_5+반성적_2+반성적_3+건강상태_2+건강상태_3+건강상태_4+건강상태_5+생활수준_2+생활수준_3+생활수준_4+생활수준_5
               
               학습태도 ~ m11*자기통제력 + m12*스트레스 + n1*인터넷사용+성별_2+학년_3+학년_4+학년_5+학년_6+거주지_2+거주지_3+거주지_4+부모_2+부모_3+부모_4+부모직장생활_2+부모직장생활_3+부모직장생활_4+인터넷사용시간_2+인터넷사용시간_3+인터넷사용시간_4+인터넷사용시간_5+인터넷사용목적_2+인터넷사용목적_3+인터넷사용목적_4+인터넷사용목적_5+반성적_2+반성적_3+건강상태_2+건강상태_3+건강상태_4+건강상태_5+생활수준_2+생활수준_3+생활수준_4+생활수준_5
               사회적관계 ~ m21*자기통제력 + m22*스트레스 + n2*인터넷사용+성별_2+학년_3+학년_4+학년_5+학년_6+거주지_2+거주지_3+거주지_4+부모_2+부모_3+부모_4+부모직장생활_2+부모직장생활_3+부모직장생활_4+인터넷사용시간_2+인터넷사용시간_3+인터넷사용시간_4+인터넷사용시간_5+인터넷사용목적_2+인터넷사용목적_3+인터넷사용목적_4+인터넷사용목적_5+반성적_2+반성적_3+건강상태_2+건강상태_3+건강상태_4+건강상태_5+생활수준_2+생활수준_3+생활수준_4+생활수준_5
               삶의질 ~ m31*자기통제력 + m32*스트레스 + n3*인터넷사용+성별_2+학년_3+학년_4+학년_5+학년_6+거주지_2+거주지_3+거주지_4+부모_2+부모_3+부모_4+부모직장생활_2+부모직장생활_3+부모직장생활_4+인터넷사용시간_2+인터넷사용시간_3+인터넷사용시간_4+인터넷사용시간_5+인터넷사용목적_2+인터넷사용목적_3+인터넷사용목적_4+인터넷사용목적_5+반성적_2+반성적_3+건강상태_2+건강상태_3+건강상태_4+건강상태_5+생활수준_2+생활수준_3+생활수준_4+생활수준_5
               
               학습태도_인터넷사용.DE:=n1
               학습태도_인터넷사용.IE:=a1*m11+a2*m12
               학습태도_인터넷사용.TE:=n1+a1*m11+a2*m12
               학습태도_자기통제력.DE:=m11
               학습태도_자기통제력.IE:=m11
               학습태도_자기통제력.TE:=m11
               학습태도_스트레스.DE:=m12
               학습태도_스트레스.IE:=m12
               학습태도_스트레스.TE:=m12
               
               사회적관계_인터넷사용.DE:=n2
               사회적관계_인터넷사용.IE:=a1*m21+a2*m22
               사회적관계_인터넷사용.TE:=n2+a1*m21+a2*m22
               사회적관계_자기통제력.DE:=m21
               사회적관계_자기통제력.IE:=m21
               사회적관계_자기통제력.TE:=m21
               사회적관계_스트레스.DE:=m22
               사회적관계_스트레스.IE:=m22
               사회적관계_스트레스.TE:=m22
               
               삶의질_인터넷사용.DE:=n3
               삶의질_인터넷사용.IE:=a1*m31+a2*m32
               삶의질_인터넷사용.TE:=n3+a1*m31+a2*m32
               삶의질_자기통제력.DE:=m31
               삶의질_자기통제력.IE:=m31
               삶의질_자기통제력.TE:=m31
               삶의질_스트레스.DE:=m32
               삶의질_스트레스.IE:=m32
               삶의질_스트레스.TE:=m32
               ', data=data, std.lv=TRUE)
result$sem <- summary(fit$sem, standardized=T, fit.measures=T, rsq=T)

(result$goodness <- fitMeasures(fit$sem, fit.measures="all") %>% data.frame() %>% rename(value=1) %>% mutate_if(is.numeric, round, 3) %>% 
    filter(grepl("chisq|df|pvalue|cfi|tli|gfi|rmsea|srmr", rownames(.))) %>% t() %>% data.frame() %>% 
    mutate(CMIN=chisq/df) %>% dplyr::select(chisq, df, pvalue, gfi, rmsea, srmr, cfi, tli, CMIN) %>% 
    rename(chisq=1, df=2, pvalue=3, GFI=4, RMSEA=5, SRMR=6, CFI=7, TLI=8)) 
clipr::write_clip(result$goodness)

print(result$regression <- result$sem$pe %>% filter(op=="~") %>% 
        dplyr::select(Dependent=lhs, Independent=rhs, Beta=std.all, Z=z, pvalue) %>% 
        mutate(stars=ifelse(pvalue < 0.001, "***", ifelse(pvalue < 0.01, "**", ifelse(pvalue < 0.05, "*", "")))) %>% 
        mutate_if(is.numeric, round, 3) %>% mutate(Dependent=str_remove(Dependent, "0")) %>% 
        mutate(Independent=str_remove(Independent, "0")))
clipr::write_clip(result$regression)

print(result$effect <- result$sem$pe %>% filter(op==":=" | op=="r2") %>% 
        mutate(stars=ifelse(pvalue < 0.001, "***",
                            ifelse(pvalue < 0.01, "**",
                                   ifelse(pvalue < 0.05, "*", "")))) %>% 
        mutate_if(is.numeric, round, 3)) 

print(result$DIeffect <- data.frame(unique(do.call(rbind.data.frame, strsplit(result$effect$label, split="\\."))[1]), 
                                    result$effect[grepl('DE', result$effect$label),] %>% dplyr::select("std.all"),
                                    result$effect[grepl('DE', result$effect$label),] %>% dplyr::select("stars"),
                                    result$effect[grepl('IE', result$effect$label),] %>% dplyr::select("std.all"),
                                    result$effect[grepl('IE', result$effect$label),] %>% dplyr::select("stars"),
                                    result$effect[grepl('TE', result$effect$label),] %>% dplyr::select("std.all"), 
                                    result$effect[grepl('TE', result$effect$label),] %>% dplyr::select("stars")) %>% 
        rename("예측변수"=1, "직접"=2, "별1"=3, "간접"=4, "별2"=5, "총"=6, "별3"=7))

result$DIeffect$직접효과 <- paste0(result$DIeffect$직접, result$DIeffect$별1)
result$DIeffect$간접효과 <- paste0(result$DIeffect$간접, result$DIeffect$별2)
result$DIeffect$총효과 <- paste0(result$DIeffect$총, result$DIeffect$별3)

print(result$DIeffect <- result$DIeffect %>% dplyr::select(예측변수, 직접효과, 간접효과, 총효과))
result$DIeffect %>% separate(예측변수, into=c("내생변수", "예측변수"), sep="_") %>% clipr::write_clip()
